# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

<b>Задание</b>

1. Загрузите датасет.


2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text
import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

In [2]:
sms_df = pd.read_table('SMSSpamCollection.txt', names=['type', 'SMS'])
sms_df.head(10)

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,type,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [3]:
sms_array = sms_df['SMS'].values
type_array = sms_df['type'].values
mark_array = np.zeros((len(type_array)))
mark_array[np.where(type_array == 'spam')] = 1

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
len(X)

5572

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [5]:
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9326402983610631

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB";
    
    "FreeMsg: Txt: claim your reward of 3 hours talk time";
    
    "Have you visited the last lecture on physics?";
    
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$";
    
    "Only 99$".
    
    Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [6]:
estimator.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [7]:
new_sms_array = np.array(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                          "FreeMsg: Txt: claim your reward of 3 hours talk time",
                          "Have you visited the last lecture on physics?",
                          "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                          "Only 99$"])
new_X = vectorizer.transform(new_sms_array).toarray()

In [8]:
estimator.predict(new_X)

array([1., 1., 0., 0., 0.])

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [9]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.8224220664187133

In [10]:
vectorizer = CountVectorizer(ngram_range=(3, 3))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.7250161555467377

In [11]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.9251382558648837

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

    По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  

In [12]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = MultinomialNB()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.9277303556851543

In [13]:
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = MultinomialNB()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.6455015177985443

In [14]:
vectorizer = CountVectorizer(ngram_range=(3, 3))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = MultinomialNB()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.37871948524573595

In [15]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = MultinomialNB()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.8884859656061002

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [16]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.8528599554172456

In [17]:
vectorizer = TfidfVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.3445904447864354

In [18]:
vectorizer = TfidfVectorizer(ngram_range=(3, 3))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.16557908589889503

In [19]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(sms_array).toarray()
y = mark_array
estimator = LogisticRegression()
cross_val_score(estimator, X, y, scoring='f1', cv=10).mean()

0.6483673712277025